<a href="https://colab.research.google.com/github/SelenaNahra/DL/blob/main/HW5P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# English to French dataset
english_to_french = [
    ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur")
]
# Special tokens for the start and end of sequences
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token
max_length = 12

# Preparing the character to index mapping and vice versa for English and French
def build_vocab(sentences):
    vocab = set()
    for pair in sentences:
        english_sentence, french_sentence = pair
        for word in english_sentence.split():
            vocab.add(word)
        for word in french_sentence.split():
            vocab.add(word)
    return vocab

english_vocab = build_vocab(english_to_french)
french_vocab = english_vocab

char_to_index_english = {"SOS": SOS_token, "EOS": EOS_token, **{char: i+2 for i, char in enumerate(sorted(list(english_vocab)))}}
index_to_char_english = {i: char for char, i in char_to_index_english.items()}

char_to_index_french = {"SOS": SOS_token, "EOS": EOS_token, **{char: i+2 for i, char in enumerate(sorted(list(french_vocab)))}}
index_to_char_french = {i: char for char, i in char_to_index_french.items()}

class EnglishFrenchDataset(Dataset):
    """Custom Dataset class for handling English-French sentence pairs."""
    def __init__(self, dataset, char_to_index_english, char_to_index_french):
        self.dataset = dataset
        self.char_to_index_english = char_to_index_english
        self.char_to_index_french = char_to_index_french

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        english_sentence, french_sentence = self.dataset[idx]
        english_tensor = torch.tensor([self.char_to_index_english[word] for word in english_sentence.split()] + [EOS_token], dtype=torch.long)
        french_tensor = torch.tensor([self.char_to_index_french[word] for word in french_sentence.split()] + [EOS_token], dtype=torch.long)
        return english_tensor, french_tensor

english_french_dataset = EnglishFrenchDataset(english_to_french, char_to_index_english, char_to_index_french)
dataloader = DataLoader(english_french_dataset, batch_size=1, shuffle=True)

# Define the Transformer model
class TranslatorTransformer(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers, nhead):
        super(TranslatorTransformer, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output)
        return output

# Hyperparameters
hidden_size = 128
num_layers = 3
nhead = 2
learning_rate = 0.0001
epochs = 100

# Model, loss, and optimizer
model = TranslatorTransformer(len(english_vocab) + 2, len(french_vocab) + 2, hidden_size, num_layers, nhead)  # Add 2 for SOS and EOS tokens
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding index
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for english_tensor, french_tensor in dataloader:
        optimizer.zero_grad()
        output = model(english_tensor)

        # Adjust the target tensor to include the whole french_tensor sequence but ensure it matches output length
        target_tensor = torch.cat((torch.ones_like(french_tensor[:, :1]) * SOS_token, french_tensor), dim=1)

        # Ensure the target tensor matches the output size if it's longer
        if target_tensor.size(1) > output.size(1):
            target_tensor = target_tensor[:, :output.size(1)]

        # Loss calculation - ensure target tensor is correct size
        loss = criterion(output.transpose(1, 2), target_tensor)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        average_loss = total_loss / len(dataloader)
        print(f'Epoch {epoch + 1}, Training Loss: {average_loss:.4f}')

def translate_english_to_french(model, eng_sentence):
    model.eval()
    with torch.no_grad():
        eng_tensor = torch.tensor([char_to_index_english[word] for word in eng_sentence.split()] + [EOS_token], dtype=torch.long)
        output = model(eng_tensor.unsqueeze(0))  # Unsqueeze to add batch dimension
        _, predicted_indices = torch.max(output, dim=2)
        predicted_words = []
        for idx in predicted_indices.squeeze():
            if idx == EOS_token:
                break
            predicted_words.append(index_to_char_french[idx.item()])
        return ' '.join(predicted_words)

def evaluate_and_show_examples(model, dataloader, criterion, n_examples=5):
    model.eval()
    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor_english, target_tensor_french) in enumerate(dataloader):
            output = model(input_tensor_english)
            target_tensor = torch.cat((torch.full((target_tensor_french.size(0), 1), SOS_token, dtype=torch.long), target_tensor_french), dim=1)

            if target_tensor.size(1) > output.size(1):
                target_tensor = target_tensor[:, :output.size(1)]

            loss = criterion(output.transpose(1, 2), target_tensor)
            total_loss += loss.item()

            _, predicted_indices = torch.max(output, dim=2)
            correct_predictions += (predicted_indices == target_tensor).all(dim=1).sum().item()
            total_sentences += target_tensor.size(0)

            if i < n_examples:
                predicted_words = [index_to_char_french.get(idx.item(), '<UNK>') for idx in predicted_indices[0] if idx not in (SOS_token, EOS_token)]
                target_words = [index_to_char_french.get(idx.item(), '<UNK>') for idx in target_tensor_french[0] if idx not in (SOS_token, EOS_token)]
                input_words = [index_to_char_english.get(idx.item(), '<UNK>') for idx in input_tensor_english[0] if idx not in (SOS_token, EOS_token)]
                print(f'Input: {" ".join(input_words)}, Target: {" ".join(target_words)}, Predicted: {" ".join(predicted_words)}')

        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}')

# If using GPU, make sure your model is transferred to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

evaluate_and_show_examples(model, dataloader, criterion, n_examples=5)

Epoch 10, Training Loss: 2.6836
Epoch 20, Training Loss: 1.9385
Epoch 30, Training Loss: 1.7398
Epoch 40, Training Loss: 1.6457
Epoch 50, Training Loss: 1.5904
Epoch 60, Training Loss: 1.5695
Epoch 70, Training Loss: 1.5399
Epoch 80, Training Loss: 1.5412
Epoch 90, Training Loss: 1.5087
Epoch 100, Training Loss: 1.5123
Input: You are tired, Target: Tu es fatigué, Predicted: la Nous es
Input: He waits for the bus, Target: Il attend le bus, Predicted: la Il attend dans bus
Input: He paints landscapes, Target: Il peint des paysages, Predicted: la Il peint
Input: He writes a letter, Target: Il écrit une lettre, Predicted: la Il construisons une
Input: She dances at the party, Target: Elle danse à la fête, Predicted: Ils Elle danse dans la
Evaluation Loss: 1.4244, Accuracy: 0.0000


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# English to French dataset
english_to_french = [
    ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur")
]
# Special tokens for the start and end of sequences
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token
max_length = 12

# Preparing the character to index mapping and vice versa for English and French
def build_vocab(sentences):
    vocab = set()
    for pair in sentences:
        english_sentence, french_sentence = pair
        for word in english_sentence.split():
            vocab.add(word)
        for word in french_sentence.split():
            vocab.add(word)
    return vocab

english_vocab = build_vocab(english_to_french)
french_vocab = english_vocab

char_to_index_english = {"SOS": SOS_token, "EOS": EOS_token, **{char: i+2 for i, char in enumerate(sorted(list(english_vocab)))}}
index_to_char_english = {i: char for char, i in char_to_index_english.items()}

char_to_index_french = {"SOS": SOS_token, "EOS": EOS_token, **{char: i+2 for i, char in enumerate(sorted(list(french_vocab)))}}
index_to_char_french = {i: char for char, i in char_to_index_french.items()}

class EnglishFrenchDataset(Dataset):
    """Custom Dataset class for handling English-French sentence pairs."""
    def __init__(self, dataset, char_to_index_english, char_to_index_french):
        self.dataset = dataset
        self.char_to_index_english = char_to_index_english
        self.char_to_index_french = char_to_index_french

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        english_sentence, french_sentence = self.dataset[idx]
        english_tensor = torch.tensor([self.char_to_index_english[word] for word in english_sentence.split()] + [EOS_token], dtype=torch.long)
        french_tensor = torch.tensor([self.char_to_index_french[word] for word in french_sentence.split()] + [EOS_token], dtype=torch.long)
        return english_tensor, french_tensor

english_french_dataset = EnglishFrenchDataset(english_to_french, char_to_index_english, char_to_index_french)
dataloader = DataLoader(english_french_dataset, batch_size=1, shuffle=True)

# Define the Transformer model
class TranslatorTransformer(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers, nhead):
        super(TranslatorTransformer, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output)
        return output

# Hyperparameters
hidden_size = 128
num_layers = 5
nhead = 4
learning_rate = 0.0001
epochs = 100

# Model, loss, and optimizer
model = TranslatorTransformer(len(english_vocab) + 2, len(french_vocab) + 2, hidden_size, num_layers, nhead)  # Add 2 for SOS and EOS tokens
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding index
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for english_tensor, french_tensor in dataloader:
        optimizer.zero_grad()
        output = model(english_tensor)

        # Adjust the target tensor to include the whole french_tensor sequence but ensure it matches output length
        target_tensor = torch.cat((torch.ones_like(french_tensor[:, :1]) * SOS_token, french_tensor), dim=1)

        # Ensure the target tensor matches the output size if it's longer
        if target_tensor.size(1) > output.size(1):
            target_tensor = target_tensor[:, :output.size(1)]

        # Loss calculation - ensure target tensor is correct size
        loss = criterion(output.transpose(1, 2), target_tensor)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        average_loss = total_loss / len(dataloader)
        print(f'Epoch {epoch + 1}, Training Loss: {average_loss:.4f}')

def translate_english_to_french(model, eng_sentence):
    model.eval()
    with torch.no_grad():
        eng_tensor = torch.tensor([char_to_index_english[word] for word in eng_sentence.split()] + [EOS_token], dtype=torch.long)
        output = model(eng_tensor.unsqueeze(0))  # Unsqueeze to add batch dimension
        _, predicted_indices = torch.max(output, dim=2)
        predicted_words = []
        for idx in predicted_indices.squeeze():
            if idx == EOS_token:
                break
            predicted_words.append(index_to_char_french[idx.item()])
        return ' '.join(predicted_words)

def evaluate_and_show_examples(model, dataloader, criterion, n_examples=5):
    model.eval()
    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor_english, target_tensor_french) in enumerate(dataloader):
            output = model(input_tensor_english)
            target_tensor = torch.cat((torch.full((target_tensor_french.size(0), 1), SOS_token, dtype=torch.long), target_tensor_french), dim=1)

            if target_tensor.size(1) > output.size(1):
                target_tensor = target_tensor[:, :output.size(1)]

            loss = criterion(output.transpose(1, 2), target_tensor)
            total_loss += loss.item()

            _, predicted_indices = torch.max(output, dim=2)
            correct_predictions += (predicted_indices == target_tensor).all(dim=1).sum().item()
            total_sentences += target_tensor.size(0)

            if i < n_examples:
                predicted_words = [index_to_char_french.get(idx.item(), '<UNK>') for idx in predicted_indices[0] if idx not in (SOS_token, EOS_token)]
                target_words = [index_to_char_french.get(idx.item(), '<UNK>') for idx in target_tensor_french[0] if idx not in (SOS_token, EOS_token)]
                input_words = [index_to_char_english.get(idx.item(), '<UNK>') for idx in input_tensor_english[0] if idx not in (SOS_token, EOS_token)]
                print(f'Input: {" ".join(input_words)}, Target: {" ".join(target_words)}, Predicted: {" ".join(predicted_words)}')

        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}')

# If using GPU, make sure your model is transferred to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Call the evaluate function
evaluate_and_show_examples(model, dataloader, criterion, n_examples=5)

Epoch 10, Training Loss: 2.6106
Epoch 20, Training Loss: 1.9636
Epoch 30, Training Loss: 1.7567
Epoch 40, Training Loss: 1.6537
Epoch 50, Training Loss: 1.6253
Epoch 60, Training Loss: 1.5812
Epoch 70, Training Loss: 1.5565
Epoch 80, Training Loss: 1.5593
Epoch 90, Training Loss: 1.5230
Epoch 100, Training Loss: 1.5045
Input: You are tired, Target: Tu es fatigué, Predicted: à Nous es
Input: She dreams of flying, Target: Elle rêve de voler, Predicted: un Elle rêve de
Input: We love music, Target: Nous aimons la musique, Predicted: la Nous aimons
Input: He paints landscapes, Target: Il peint des paysages, Predicted: Le Il peint
Input: The cat is sleeping, Target: Le chat dort, Predicted: sont Le Il dort
Evaluation Loss: 1.4212, Accuracy: 0.0000


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# English to French dataset
english_to_french = [
    ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur")
]
# Special tokens for the start and end of sequences
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token
max_length = 12

# Preparing the character to index mapping and vice versa for English and French
def build_vocab(sentences):
    vocab = set()
    for pair in sentences:
        english_sentence, french_sentence = pair
        for word in english_sentence.split():
            vocab.add(word)
        for word in french_sentence.split():
            vocab.add(word)
    return vocab

english_vocab = build_vocab(english_to_french)
french_vocab = english_vocab

char_to_index_english = {"SOS": SOS_token, "EOS": EOS_token, **{char: i+2 for i, char in enumerate(sorted(list(english_vocab)))}}
index_to_char_english = {i: char for char, i in char_to_index_english.items()}

char_to_index_french = {"SOS": SOS_token, "EOS": EOS_token, **{char: i+2 for i, char in enumerate(sorted(list(french_vocab)))}}
index_to_char_french = {i: char for char, i in char_to_index_french.items()}

class EnglishFrenchDataset(Dataset):
    """Custom Dataset class for handling English-French sentence pairs."""
    def __init__(self, dataset, char_to_index_english, char_to_index_french):
        self.dataset = dataset
        self.char_to_index_english = char_to_index_english
        self.char_to_index_french = char_to_index_french

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        english_sentence, french_sentence = self.dataset[idx]
        english_tensor = torch.tensor([self.char_to_index_english[word] for word in english_sentence.split()] + [EOS_token], dtype=torch.long)
        french_tensor = torch.tensor([self.char_to_index_french[word] for word in french_sentence.split()] + [EOS_token], dtype=torch.long)
        return english_tensor, french_tensor

english_french_dataset = EnglishFrenchDataset(english_to_french, char_to_index_english, char_to_index_french)
dataloader = DataLoader(english_french_dataset, batch_size=1, shuffle=True)

# Define the Transformer model
class TranslatorTransformer(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers, nhead):
        super(TranslatorTransformer, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output)
        return output

# Hyperparameters
hidden_size = 512
num_layers = 5
nhead = 4
learning_rate = 0.0001
epochs = 200

# Model, loss, and optimizer
model = TranslatorTransformer(len(english_vocab) + 2, len(french_vocab) + 2, hidden_size, num_layers, nhead)  # Add 2 for SOS and EOS tokens
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding index
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for english_tensor, french_tensor in dataloader:
        optimizer.zero_grad()
        output = model(english_tensor)

        # Adjust the target tensor to include the whole french_tensor sequence but ensure it matches output length
        target_tensor = torch.cat((torch.ones_like(french_tensor[:, :1]) * SOS_token, french_tensor), dim=1)

        # Ensure the target tensor matches the output size if it's longer
        if target_tensor.size(1) > output.size(1):
            target_tensor = target_tensor[:, :output.size(1)]

        # Loss calculation - ensure target tensor is correct size
        loss = criterion(output.transpose(1, 2), target_tensor)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        average_loss = total_loss / len(dataloader)
        print(f'Epoch {epoch + 1}, Training Loss: {average_loss:.4f}')

def translate_english_to_french(model, eng_sentence):
    model.eval()
    with torch.no_grad():
        eng_tensor = torch.tensor([char_to_index_english[word] for word in eng_sentence.split()] + [EOS_token], dtype=torch.long)
        output = model(eng_tensor.unsqueeze(0))  # Unsqueeze to add batch dimension
        _, predicted_indices = torch.max(output, dim=2)
        predicted_words = []
        for idx in predicted_indices.squeeze():
            if idx == EOS_token:
                break
            predicted_words.append(index_to_char_french[idx.item()])
        return ' '.join(predicted_words)

def evaluate_and_show_examples(model, dataloader, criterion, n_examples=5):
    model.eval()
    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor_english, target_tensor_french) in enumerate(dataloader):
            output = model(input_tensor_english)
            target_tensor = torch.cat((torch.full((target_tensor_french.size(0), 1), SOS_token, dtype=torch.long), target_tensor_french), dim=1)

            if target_tensor.size(1) > output.size(1):
                target_tensor = target_tensor[:, :output.size(1)]

            loss = criterion(output.transpose(1, 2), target_tensor)
            total_loss += loss.item()

            _, predicted_indices = torch.max(output, dim=2)
            correct_predictions += (predicted_indices == target_tensor).all(dim=1).sum().item()
            total_sentences += target_tensor.size(0)

            if i < n_examples:
                predicted_words = [index_to_char_french.get(idx.item(), '<UNK>') for idx in predicted_indices[0] if idx not in (SOS_token, EOS_token)]
                target_words = [index_to_char_french.get(idx.item(), '<UNK>') for idx in target_tensor_french[0] if idx not in (SOS_token, EOS_token)]
                input_words = [index_to_char_english.get(idx.item(), '<UNK>') for idx in input_tensor_english[0] if idx not in (SOS_token, EOS_token)]
                print(f'Input: {" ".join(input_words)}, Target: {" ".join(target_words)}, Predicted: {" ".join(predicted_words)}')

        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}')

evaluate_and_show_examples(model, dataloader, criterion, n_examples=5)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 10, Training Loss: 1.7837
Epoch 20, Training Loss: 1.6610
Epoch 30, Training Loss: 1.6107
Epoch 40, Training Loss: 1.5862
Epoch 50, Training Loss: 1.6229
Epoch 60, Training Loss: 1.5503
Epoch 70, Training Loss: 1.5274
Epoch 80, Training Loss: 1.7358
Epoch 90, Training Loss: 1.5352
Epoch 100, Training Loss: 1.5175
Epoch 110, Training Loss: 1.5148
Epoch 120, Training Loss: 1.5078
Epoch 130, Training Loss: 1.5004
Epoch 140, Training Loss: 1.5071
Epoch 150, Training Loss: 1.4915
Epoch 160, Training Loss: 1.4918
Epoch 170, Training Loss: 1.4886
Epoch 180, Training Loss: 1.4930
Epoch 190, Training Loss: 1.4817
Epoch 200, Training Loss: 1.4740


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# English to French dataset
english_to_french = [
    ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur")
]
# Special tokens for the start and end of sequences
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token
max_length = 12

# Preparing the character to index mapping and vice versa for English and French
def build_vocab(sentences):
    vocab = set()
    for pair in sentences:
        english_sentence, french_sentence = pair
        for word in english_sentence.split():
            vocab.add(word)
        for word in french_sentence.split():
            vocab.add(word)
    return vocab

english_vocab = build_vocab(english_to_french)
french_vocab = english_vocab

char_to_index_english = {"SOS": SOS_token, "EOS": EOS_token, **{char: i+2 for i, char in enumerate(sorted(list(english_vocab)))}}
index_to_char_english = {i: char for char, i in char_to_index_english.items()}

char_to_index_french = {"SOS": SOS_token, "EOS": EOS_token, **{char: i+2 for i, char in enumerate(sorted(list(french_vocab)))}}
index_to_char_french = {i: char for char, i in char_to_index_french.items()}

class EnglishFrenchDataset(Dataset):
    """Custom Dataset class for handling English-French sentence pairs."""
    def __init__(self, dataset, char_to_index_english, char_to_index_french):
        self.dataset = dataset
        self.char_to_index_english = char_to_index_english
        self.char_to_index_french = char_to_index_french

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        english_sentence, french_sentence = self.dataset[idx]
        english_tensor = torch.tensor([self.char_to_index_english[word] for word in english_sentence.split()] + [EOS_token], dtype=torch.long)
        french_tensor = torch.tensor([self.char_to_index_french[word] for word in french_sentence.split()] + [EOS_token], dtype=torch.long)
        return english_tensor, french_tensor

english_french_dataset = EnglishFrenchDataset(english_to_french, char_to_index_english, char_to_index_french)
dataloader = DataLoader(english_french_dataset, batch_size=1, shuffle=True)

# Define the Transformer model
class TranslatorTransformer(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers, nhead):
        super(TranslatorTransformer, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output)
        return output

# Hyperparameters
hidden_size = 512
num_layers = 5
nhead = 4
learning_rate = 0.0001
epochs = 200

# Model, loss, and optimizer
model = TranslatorTransformer(len(english_vocab) + 2, len(french_vocab) + 2, hidden_size, num_layers, nhead)  # Add 2 for SOS and EOS tokens
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding index
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def translate_english_to_french(model, eng_sentence):
    model.eval()
    with torch.no_grad():
        eng_tensor = torch.tensor([char_to_index_english[word] for word in eng_sentence.split()] + [EOS_token], dtype=torch.long)
        output = model(eng_tensor.unsqueeze(0))  # Unsqueeze to add batch dimension
        _, predicted_indices = torch.max(output, dim=2)
        predicted_words = []
        for idx in predicted_indices.squeeze():
            if idx == EOS_token:
                break
            predicted_words.append(index_to_char_french[idx.item()])
        return ' '.join(predicted_words)

def evaluate_and_show_examples(model, dataloader, criterion, n_examples=5):
    model.eval()
    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor_english, target_tensor_french) in enumerate(dataloader):
            output = model(input_tensor_english)
            target_tensor = torch.cat((torch.full((target_tensor_french.size(0), 1), SOS_token, dtype=torch.long), target_tensor_french), dim=1)

            if target_tensor.size(1) > output.size(1):
                target_tensor = target_tensor[:, :output.size(1)]

            loss = criterion(output.transpose(1, 2), target_tensor)
            total_loss += loss.item()

            _, predicted_indices = torch.max(output, dim=2)
            correct_predictions += (predicted_indices == target_tensor).all(dim=1).sum().item()
            total_sentences += target_tensor.size(0)

            if i < n_examples:
                predicted_words = [index_to_char_french.get(idx.item(), '<UNK>') for idx in predicted_indices[0] if idx not in (SOS_token, EOS_token)]
                target_words = [index_to_char_french.get(idx.item(), '<UNK>') for idx in target_tensor_french[0] if idx not in (SOS_token, EOS_token)]
                input_words = [index_to_char_english.get(idx.item(), '<UNK>') for idx in input_tensor_english[0] if idx not in (SOS_token, EOS_token)]
                print(f'Input: {" ".join(input_words)}, Target: {" ".join(target_words)}, Predicted: {" ".join(predicted_words)}')

        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}')

evaluate_and_show_examples(model, dataloader, criterion, n_examples=5)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Input: They visit the Eiffel Tower, Target: Ils visitent la tour Eiffel, Predicted: starts parlons lire sings pour university
Input: I am cold, Target: J'ai froid, Predicted: movie J'ai fatigué university
Input: We watch movies on Fridays, Target: Nous regardons des films le vendredi, Predicted: un Tu soir fleurs oiseaux university
Input: The rain falls gently, Target: La pluie tombe doucement, Predicted: l'université drives speaks bloom university
Input: We are friends, Target: Nous sommes amis, Predicted: un radio shining university
Evaluation Loss: 6.1707, Accuracy: 0.0000
